<div>
  <h1>
    <img src="https://cdn.prod.website-files.com/66cf2bfc3ed15b02da0ca770/66d07240057721394308addd_Logo%20(1).svg" style="height: 40px; vertical-align: bottom;" />
    +
    <img src="https://kumo-ai.github.io/kumo-sdk/docs/_static/kumo-logo.svg" style="height: 40px; vertical-align: bottom;""/>
    RFM E-Commerce Churn & Recovery Demo
  </h1>
</div>

This notebook demonstrates how to combine **CrewAI agents** with the **`KumoRFM` MCP server** to implement a complete e-commerce churn and recovery workflow.  

The workflow includes:

1. **Data Ingestion & Graph Materialization**  
   Load S3 Parquet files into a `KumoRFM` graph, validate schemas, and summarize the resulting graph.

2. **Candidate Selection**  
   Identify the top-100 users who have recently returned items, serving as the target audience for churn prediction.

3. **Churn Prediction**  
   Compute churn probabilities for the selected users using predictive queries via `KumoRFM`, highlighting high-risk users.

4. **Item Recommendation**  
   Generate personalized item recommendations for the high-risk users, leveraging predictive queries and table lookups.

5. **Re-Engagement Email Drafting**  
   Automatically draft personalized, persuasive emails including recommended items and optional discount codes to encourage user retention.

This demo illustrates the **full cycle of intelligent agent orchestration** for data engineering, analytics, and customer engagement using CrewAI and MCP tools.

## Running this notebook
1. Create a new environment with `uv` or `pip` with python >= 3.10
2. Install required packages:
```
# Install required libraries
pip install kumo-rfm-mcp 'crewai-tools[mcp]' s3fs jupyter
```
3. (Optional) Obtain the Kumo API key at [kumorfm.ai](https://kumorfm.ai/api-keys) and set it as an environment variable `export KUMO_API_KEY=<your-api-key>`
4. Run the notebook via `jupyter notebook`

## Installation

We start by installing the necessary packages, and setting up API keys, both for `KumoRFM` and OpenAI:

In [12]:
!pip install kumo-rfm-mcp 'crewai-tools[mcp]' s3fs


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


If you do not have a `KumoRFM` API key already, this workflow will automatically generate one for you for free:

In [ ]:
import os
from kumoai.experimental import rfm

if not os.environ.get("KUMO_API_KEY"):
    rfm.authenticate()

# or for manual setup:
# os.environ['KUMO_API_KEY'] = "<YOUR_KUMO_API_KEY>"

In order to use crewAI agents, we need to set up our LLM provider.
In this example, we will use `openai/o4-mini`.
In order to aquire an OpenAI API key, go to https://platform.openai.com/api-keys.
Afterwards, paste the key into the text field below:

In [16]:
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key:  ········


## Data Loading

We are operating on an e-commerce datasets, which includes `users`, `items`, `views`, `orders` and `returns`.
We can access this data from S3:

In [4]:
root = 's3://kumo-sdk-public/rfm-datasets/ecom'
path_dict = {
    'users': f'{root}/users.parquet',
    'items': f'{root}/items.parquet',
    'views': f'{root}/views.parquet',
    'orders': f'{root}/orders.parquet',
    'returns': f'{root}/returns.parquet',
}

## MCP Server Setup

We are now ready to set up our **`KumoRFM` MCP server**.
This server will run locally and can be started via `python -m kumo_rfm_mcp.server`.
We additionally pass our previously aquired `KUMO_API_KEY` as environment variable.

In [5]:
from mcp import StdioServerParameters

params = StdioServerParameters(
    command='python',
    args=['-m', 'kumo_rfm_mcp.server'],
    env={'KUMO_API_KEY': os.environ['KUMO_API_KEY']},
)

## Agentic Workflow

Let's recall our desired agentic workflow:

We want to predict churn risk for users that have previously returned an item, and want to draft a persuasive mail with personalized recommendations.

For this, we start by ingesting the `returns.parquet` data from S3 and selecting the top-100 users who have most recently returned items.  
This subset will serve as the target audience for our churn prediction and re-engagement workflow.

In [17]:
import pandas as pd

df = pd.read_parquet(path_dict['returns'], columns=['user_id', 'date'])
df = df.sort_values('date', ascending=False)
users = df['user_id'].drop_duplicates().head(100).tolist()

Finally, we are ready to spin up our crewAI workflow:
1. We initialize two agents that can utilize the tools provided by `KumoRFM`. One agent will be responsible for validation our data, inter-connecting tables, and setting up our graph. The other agent will be responsible for querying `KumoRFM` to obtain predictions.
2. We use these agents for several tasks, including **(1)** graph setup, **(2)** churn prediction, **(3)** item recommendation, and **(4)** creative writing.
3. We bundle these tasks together into a sequential workflow.

In [21]:
from crewai import Agent, Crew, Process, Task
from crewai_tools import MCPServerAdapter

with MCPServerAdapter(params) as tools:
    # Create a CrewAI agent specialized in data engineering.
    # We provide it with tools from the MCP server to perform documentation lookup,
    # table inspection, metadata updates, and graph materialization.
    graph_agent = Agent(
        role="Graph Architect",
        goal=("Ingest S3 Parquet files, verify schema, and materialize a "
              "KumoRFM graph"),
        backstory=(
            "I am a meticulous data engineer specialising in schema design "
            "and reliable data materialization for RFM workloads. I validate "
            "inputs, ensure keys and time columns are correct, and produce a "
            "compact schema summary."),
        tools=[
            tools['get_docs'],
            tools['inspect_table_files'],
            tools['update_graph_metadata'],
            tools['materialize_graph'],
        ],
        reasoning=False,
        verbose=True,
    )
    graph_task = Task(
        description=(
            f"Create and materialize a KumoRFM graph from the following S3 "
            f"files:\n\n{path_dict}\n\nMake sure that links between tables "
            f"are set up properly. Before making tool calls, consult the "
            f"graph-setup documentation (get_docs)"),
        expected_output="A summary of the graph schema",
        agent=graph_agent,
        markdown=True,
    )

    # Next, we define a predictive agent whose responsibility is to query KumoRFM:  
    predict_agent = Agent(
        role="Data Scientist",
        goal=("Produce robust churn predictions and high-quality item "
              "recommendations"),
        backstory=("I write predictive queries against KumoRFM, and return "
                   "concise, auditable results."),
        tools=[
            tools['get_docs'],
            tools['predict'],
            tools['lookup_table_rows'],
        ],
        reasoning=False,
        verbose=True,
    )
    churn_task = Task(
        description=(
            f"For the provided candidate users, compute their churn "
            f"probability.\n\nUsers: {users}\n\nRequirements:\n"
            f"- Churn := likelihood of zero orders in the next 30 days\n"
            f"- Use the latest available data and appropriate predictive "
            f"query primitives\n"
            f"- Run the prediction in a batch for all users\n\n"
            f"Before making tool calls, consult the predictive-query "
            f"documentation (get_docs)"),
        expected_output=("Top-2 high-risk users"),
        agent=predict_agent,
        markdown=True,
    )
    rec_task = Task(
        description=(
            "For the top-2 high-risk users produced by the churn task, "
            "recommend 3 novel items per user.\n\nRequirements:\n"
            "- Predict which item IDs a user will order\n"
            "- Use the 'LIST_DISTINCT' query on the 'orders.item_id' column\n"
            "- Use a time range of 30 days into the future\n"
            "- Rank the top 3 most likely items\n"
            "- Use the latest available data and appropriate predictive "
            "query primitives\n"
            "- Run the prediction in a batch for all users\n"
            "- Use the 'lookup_table_rows' tool for recommended item metadata "
            "retrieval on the 'items' table\n\n"
            "Before making tool calls, consult the predictive-query "
            "documentation (get_docs)"),
        expected_output=("A summary of the recommended items with detailed "
                         "item descriptions"),
        agent=predict_agent,
        markdown=True,
    )

    mail_task = Task(
        description=(
            "Draft a short, persuasive re-engagement email for each high-risk "
            "user.\n\nRequirements:\n"
            "- Include a subject line, plain-text body, and a suggested "
            "discount code (if appropriate)\n"
            "- Highlight that the discount is offered as compensation for the "
            "returned orders\n"
            "- Include the recommendations in the body"),
        expected_output=("List of personalized email drafts"),
        agent=predict_agent,
        markdown=True,
    )

    # We assemble all agents and tasks into a Crew and execute the workflow sequentially.  
    # This produces a full pipeline from data ingestion, graph materialization,
    # churn prediction, item recommendation, to email drafting.
    crew = Crew(
        agents=[graph_agent, predict_agent],
        tasks=[graph_task, churn_task, rec_task, mail_task],
        verbose=True,
        process=Process.sequential,
    )

    result = crew.kickoff()

/Users/rusty1s/miniconda3/lib/python3.10/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'items', 'anyOf', 'enum', 'properties'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 27c3801f-d2d5-4bf2-87b7-ff8a6ac57f85                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Graph Architect                                                                                         │
│                                                                                                                 │
│  Task: Create and materialize a KumoRFM graph from the following S3 files:                                      │
│                                                                                                                 │
│  {'users': 's3://kumo-sdk-public/rfm-datasets/ecom/users.parquet', 'items':                                     │
│  's3://kumo-sdk-public/rfm-datasets/ecom/items.parquet', 'views':                                               │
│  's3://kumo-sdk-public/rfm-datasets/ecom/views.parquet', 'orders':                                              │
│  's3://kumo-sdk-public/rfm-datasets/ecom/orders.parquet', 'returns':                                            │
│  's3://kumo-sdk-public/rfm-datasets/ecom/returns.parquet'}                                                      │
│                                                                                                                 │
│  Make sure that links between tables are set up properly. Before making tool calls, consult the graph-setup     │
│  documentation (get_docs)                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Graph Architect                                                                                         │
│                                                                                                                 │
│  Using Tool: get_docs                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  {"rows":[{"user_id":11460,"fn":false,"active":false,"club_member_status":"ACTIVE","fashion_news_frequency":"N  │
│  one","age":30},{"user_id":11099,"fn":false,"active":false,"club_member_status":"ACTIVE","fashion_news_frequen  │
│  cy":"None","age":37}]}                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Scientist                                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Re-engagement Email Drafts                                                                                   │
│                                                                                                                 │
│  ## Email for User ID: 11460                                                                                    │
│  **Subject Line:** We Miss You! Enjoy a Special Discount on Us 🌊                                               │
│                                                                                                                 │
│  **Dear Valued Customer,**                                                                                      │
│                                                                                                                 │
│  We hope this message finds you well! We noticed that it has been a while since your last order, and we truly   │
│  miss having you as part of our community. As a token of our appreciation, we would like to offer you a **10%   │
│  discount** on your next purchase to help make your shopping experience even better.                            │
│                                                                                                                 │
│  Code: `WELCOMEBACK10`                                                                                          │
│                                                                                                                 │
│  We understand that sometimes things don’t go as planned. We noticed that you returned a couple of orders, and  │
│  we want to ensure you're completely satisfied with your purchases. Here are some *fantastic items* we've       │
│  selected just for you:                                                                                         │
│                                                                                                                 │
│  - **Timeless Triangle Top**                                                                                    │
│    - *Product Type:* Bikini top                                                                                 │
│    - *Color:* Light Blue                                                                                        │
│    - *Description:* Lined triangle bikini top with adjustable shoulder straps that can be fastened in           │
│  different ways.                                                                                                │
│                                                                                                                 │
│  - **Lazer Razer Brief**                                                                                        │
│    - *Product Type:* Swimwear bottom                                                                            │
│    - *Color:* Light Blue                                                                                        │
│    - *Description:* Fully lined bikini bottoms with a mid waist and medium coverage at the back.                │
│                                                                                                                 │
│  - **Timeless Cheeky Brief**                                                                                    │
│    - *Product Type:* Swimwear bottom                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: e5d1268a-5ca5-4b99-bb57-bae5c7ecfb21                                                                     │
│  Agent: Data Scientist                                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 27c3801f-d2d5-4bf2-87b7-ff8a6ac57f85                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Re-engagement Email Drafts                                                                     │
│                                                                                                                 │
│  ## Email for User ID: 11460                                                                                    │
│  **Subject Line:** We Miss You! Enjoy a Special Discount on Us 🌊                                               │
│                                                                                                                 │
│  **Dear Valued Customer,**                                                                                      │
│                                                                                                                 │
│  We hope this message finds you well! We noticed that it has been a while since your last order, and we truly   │
│  miss having you as part of our community. As a token of our appreciation, we would like to offer you a **10%   │
│  discount** on your next purchase to help make your shopping experience even better.                            │
│                                                                                                                 │
│  Code: `WELCOMEBACK10`                                                                                          │
│                                                                                                                 │
│  We understand that sometimes things don’t go as planned. We noticed that you returned a couple of orders, and  │
│  we want to ensure you're completely satisfied with your purchases. Here are some *fantastic items* we've       │
│  selected just for you:                                                                                         │
│                                                                                                                 │
│  - **Timeless Triangle Top**                                                                                    │
│    - *Product Type:* Bikini top                                                                                 │
│    - *Color:* Light Blue                                                                                        │
│    - *Description:* Lined triangle bikini top with adjustable shoulder straps that can be fastened in           │
│  different ways.                                                                                                │
│                                                                                                                 │
│  - **Lazer Razer Brief**                                                                                        │
│    - *Product Type:* Swimwear bottom                                                                            │
│    - *Color:* Light Blue                                                                                        │
│    - *Description:* Fully lined bikini bottoms with a mid waist and medium coverage at the back.                │
│                                                                                                                 │
│  - **Timeless Cheeky Brief**                           

Let's print out our final mail contents:

In [29]:
from IPython.display import Markdown

display(Markdown(result.raw))

# Re-engagement Email Drafts

## Email for User ID: 11460
**Subject Line:** We Miss You! Enjoy a Special Discount on Us 🌊

**Dear Valued Customer,**

We hope this message finds you well! We noticed that it has been a while since your last order, and we truly miss having you as part of our community. As a token of our appreciation, we would like to offer you a **10% discount** on your next purchase to help make your shopping experience even better.

Code: `WELCOMEBACK10`

We understand that sometimes things don’t go as planned. We noticed that you returned a couple of orders, and we want to ensure you're completely satisfied with your purchases. Here are some *fantastic items* we've selected just for you:

- **Timeless Triangle Top**
  - *Product Type:* Bikini top
  - *Color:* Light Blue
  - *Description:* Lined triangle bikini top with adjustable shoulder straps that can be fastened in different ways.

- **Lazer Razer Brief**
  - *Product Type:* Swimwear bottom
  - *Color:* Light Blue
  - *Description:* Fully lined bikini bottoms with a mid waist and medium coverage at the back.

- **Timeless Cheeky Brief**
  - *Product Type:* Swimwear bottom
  - *Color:* Black
  - *Description:* Fully lined bikini bottoms with a low waist for a comfortable fit.

We hope to see you soon, and can't wait to help you find something you'll love!

Warm regards,  
[Your Company Name]

---

## Email for User ID: 11099
**Subject Line:** We Want You Back! Enjoy a Special Offer 🎉

**Dear [Customer's Name],**

We noticed you haven't shopped with us in a while, and we want to change that! To welcome you back, we’re offering a **15% discount** on your next order as a little nudge.

Code: `WELCOMEHOME15`

We understand that sometimes returns happen, and it might not have been the right fit for you. So we've curated some *top recommendations* that we think you'll love:

- **Cool Anki Earring PK**
  - *Product Type:* Earring
  - *Color:* Gold
  - *Description:* Stunning metal earrings in various unique designs.

- **Haze LS**
  - *Product Type:* Top
  - *Color:* Light Orange
  - *Description:* Comfortable short polo-neck top in viscose jersey with long sleeves.

- **Lourdes Cable Cardigan**
  - *Product Type:* Cardigan
  - *Color:* White
  - *Description:* Soft cable knit cardigan with a deep V-neck and buttons down the front.

We hope you'll take advantage of this exclusive offer and find something you’ll cherish!

Best wishes,  
[Your Company Name]

## We'd love to hear from you! ❤️

1. **Found a bug or have a feature request?**  
   Submit issues directly on [GitHub](https://github.com/kumo-ai/kumo-rfm). Your feedback helps us improve RFM for everyone.

2. **Built something cool with RFM? We'd love to see it!**  
   Share your project on LinkedIn and tag @kumo. We regularly spotlight on our official channels—yours could be next!

<div align="left">
  <img src="https://kumo-sdk-public.s3.us-west-2.amazonaws.com/rfm-colabs/kumo_ai_logo.jpeg" width="30" />
</div>